In [ ]:
from data_loading import *

In [ ]:
levels = 36
variables = ["U", "V"]
compression = "zlib"
compression_level = 9
skip = True

In [ ]:
for mm in tqdm(range(1, 13)):
    for dd in tqdm(range(1, monthrange(2001, mm)[1] + 1)):
        output = f"MERRA2.tavg3_3d_asm_Nv.YAVG{mm:0>2}{dd:0>2}.nc4"

        if os.path.exists(f"YAVG/{output}") and skip:
            continue

        with open_nc4_dataset(output, folder="YAVG/", mode="w") as dst:
            dst.createDimension("time", 8)
            dst.createDimension("lev", 36)
            dst.createDimension("lat", 361)
            dst.createDimension("lon", 576 // 2)  # //2 because we pack float16s as float32s

            for variable in variables:
                dst.createVariable(variable, "f", dst.dimensions,
                                   compression=compression,
                                   complevel=compression_level)

                data = load_yavg_variable_on_day("MERRA2_{}.tavg3_3d_asm_Nv.{}mmdd.nc4".replace("mmdd", f"{mm:0>2}{dd:0>2}"),
                                                 variable)
                dst[variable][:] = data.view("float32")
